# 🧹 Data Preparation & Feature Engineering

This notebook is the **second stage** in the machine learning pipeline and builds directly upon the insights from our data exploration. Our goal here is to **prepare the dataset** for effective training by cleaning, transforming, and engineering features that enhance the learning signal.

---

### 📌 Notebook Objective

In this notebook, we aim to:
- Clean and normalize the raw dataset
- Convert categorical features to numerical format
- Handle missing values and ambiguous data entries
- Engineer useful features from existing columns
- Set up a reproducible ML preprocessing pipeline

This ensures the dataset is model-ready and consistent across experiments.

---

### 🔍 Why This Matters

Data quality and representation directly affect model performance and fairness. A well-prepared dataset:
- Improves generalization
- Prevents data leakage
- Enables fair comparison between models
- Helps downstream explainability efforts

---


## 1. Load Data & Initial Copy 📥

In [2]:
# Importing the libaries needed 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import resample  


# Setting the plot style
sns.set(style='whitegrid')


In [50]:
# Importing the dataset into a DataFrame
df = pd.read_csv('../data/diabetic_data.csv')
# Creating a copy to not modify the original dataset
df_copy = df.copy()

# Displaying the first 10 rows of the copied DataFrame
df_copy.head(10)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),?,3,1,2,4,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),?,1,1,7,5,...,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),?,2,1,4,13,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),?,3,3,4,12,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


## 2. Handle Ambiguous & Missing Values ❓

#### Cleaning placeholder values and setting them as NaN values

In [51]:
# Determening placeholder values in based on commonly used values
placeholder_values = [
    'na', 'Na', 'NA',
    'NaN', 'nan', 'NAN',
    'n/a', 'N/A', 'N\A',
    'n.a.', 'N.A.', 'n.a', 'N.A',
    '?', '-', '--', '.', '*'
]

# Converting placeholder values to NaN
df_copy = df_copy.replace(placeholder_values, np.nan)

df_copy.isnull().sum().sort_values(ascending=False)

weight                      98569
max_glu_serum               96420
A1Cresult                   84748
medical_specialty           49949
payer_code                  40256
race                         2273
diag_3                       1423
diag_2                        358
diag_1                         21
encounter_id                    0
troglitazone                    0
tolbutamide                     0
pioglitazone                    0
rosiglitazone                   0
acarbose                        0
miglitol                        0
citoglipton                     0
tolazamide                      0
examide                         0
glipizide                       0
insulin                         0
glyburide-metformin             0
glipizide-metformin             0
glimepiride-pioglitazone        0
metformin-rosiglitazone         0
metformin-pioglitazone          0
change                          0
diabetesMed                     0
glyburide                       0
repaglinide   

#### Checking The DataSet For Information

In [52]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

#### Dropping Features That Adds No Value
Based on `01_Data_exploration` there were several features in the bianry and low cardinality data subsets that are redundant, have to low variance or simply do not give any values. In addition, features with a singular class or those that are missing to many values will also be dropped.

In [53]:
# Dropping the columns found in the EDA to be unnecessary or of no value
categories_to_drop = [
    'weight', 'payer_code', 'medical_specialty',
    'max_glu_serum', 'A1Cresult',                           # Dropped due to high number of null values
    'examide', 'citoglipton',                               # Dropped due to low number of unique values/classes
    'acetohexamide', 'tolbutamide', 'troglitazone',         # --------- Binary values-----------
    'glipizide-metformin', 'glimepiride-pioglitazone',
    'metformin-rosiglitazone', 'metformin-pioglitazone',    # --- Dropped due to low variance/redudancy---
    'repaglinide', 'nateglinide', 'chlorpropamide',         # --------- Low Cardinality values-----------
    'pioglitazone', 'rosiglitazone', 'acarbose',
    'miglitol', 'tolazamide', 'glyburide-metformin',
    'glimepiride',                                          # -- Dropped due to low variance and redundancy --
]

categories_to_engineer = [
    'encounter_id', 'patient_nbr',                          # For revisit count, uniqueness
    'diag_1', 'diag_2', 'diag_3'                            # For merged diagnosis feature
]

all_colls_to_drop = categories_to_drop + categories_to_engineer

# Dropping the columns from the DataFrame
df_copy = df_copy.drop(columns=all_colls_to_drop)

# Diplaying the DataFrame after dropping the columns and its new shape 
print(df_copy.shape)
df_copy.head(10)


(101766, 21)


,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,number_emergency,number_inpatient,number_diagnoses,metformin,glipizide,glyburide,insulin,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,0,0,1,No,No,No,No,No,No,NO
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,0,0,9,No,No,No,Up,Ch,Yes,>30
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,0,1,6,No,Steady,No,No,No,Yes,NO
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,0,0,7,No,No,No,Up,Ch,Yes,NO
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,0,0,5,No,Steady,No,Steady,Ch,Yes,NO
5,Caucasian,Male,[50-60),2,1,2,3,31,6,16,...,0,0,9,No,No,No,Steady,No,Yes,>30
6,Caucasian,Male,[60-70),3,1,2,4,70,1,21,...,0,0,7,Steady,No,No,Steady,Ch,Yes,NO
7,Caucasian,Male,[70-80),1,1,7,5,73,0,12,...,0,0,8,No,No,Steady,No,No,Yes,>30
8,Caucasian,Female,[80-90),2,1,4,13,68,2,28,...,0,0,8,No,Steady,No,Steady,Ch,Yes,NO
9,Caucasian,Female,[90-100),3,3,4,12,33,3,18,...,0,0,8,No,No,No,Steady,Ch,Yes,NO


#### About the Dropped Features

Many of features were dropped in this part, mainly due to a combination of high missingness, low variance and redundancy, or lack of useful information. In addition, some features (like IDs and diagnosis codes) were set aside for later use in feature engineering (part 6).


#### Splitting the Data Into Numerical and Cateogircal Subsets
Code "borrowed" drom `01_data_exploration.upynb` for the splitting

In [54]:
# Getting the list of categorical and numerical features and storing in an array 
categorical_features = df_copy.select_dtypes(include=['object', 'bool', 'category']).columns.tolist().copy()
numerical_features = df_copy.select_dtypes(include=['int64', 'float64']).columns.tolist().copy()

# Features that have codes (categorical feature) but could be numerical in the data. from IDS_mapping.csv and 
# https://datasets.aim-ahead.net/dataset/p/UCI_DS_296
hidden_categorical_features = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']

for feature in hidden_categorical_features:
    if feature in numerical_features:
        numerical_features.remove(feature)
    if feature not in categorical_features:
        categorical_features.append(feature)

# Checking so that every feature has been acccounted for 
num_of_splitted_features = len(categorical_features) + len(numerical_features)

if num_of_splitted_features != len(df_copy.columns):
    accounted = set(numerical_features + categorical_features)
    missing = set(df_copy.columns) - accounted
    print(f'! WARNING ! Unaccounted features: {missing}')
else:
    print('All features have been accounted for')
    df_categorical = df_copy[categorical_features]
    df_numerical = df_copy[numerical_features]




All features have been accounted for


In [55]:
df_categorical.head(10)

,race,gender,age,metformin,glipizide,glyburide,insulin,change,diabetesMed,readmitted,admission_type_id,discharge_disposition_id,admission_source_id
0,Caucasian,Female,[0-10),No,No,No,No,No,No,NO,6,25,1
1,Caucasian,Female,[10-20),No,No,No,Up,Ch,Yes,>30,1,1,7
2,AfricanAmerican,Female,[20-30),No,Steady,No,No,No,Yes,NO,1,1,7
3,Caucasian,Male,[30-40),No,No,No,Up,Ch,Yes,NO,1,1,7
4,Caucasian,Male,[40-50),No,Steady,No,Steady,Ch,Yes,NO,1,1,7
5,Caucasian,Male,[50-60),No,No,No,Steady,No,Yes,>30,2,1,2
6,Caucasian,Male,[60-70),Steady,No,No,Steady,Ch,Yes,NO,3,1,2
7,Caucasian,Male,[70-80),No,No,Steady,No,No,Yes,>30,1,1,7
8,Caucasian,Female,[80-90),No,Steady,No,Steady,Ch,Yes,NO,2,1,4
9,Caucasian,Female,[90-100),No,No,No,Steady,Ch,Yes,NO,3,3,4


In [56]:
df_numerical.head(10)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,1,41,0,1,0,0,0,1
1,3,59,0,18,0,0,0,9
2,2,11,5,13,2,0,1,6
3,2,44,1,16,0,0,0,7
4,1,51,0,8,0,0,0,5
5,3,31,6,16,0,0,0,9
6,4,70,1,21,0,0,0,7
7,5,73,0,12,0,0,0,8
8,13,68,2,28,0,0,0,8
9,12,33,3,18,0,0,0,8


#### Imputating the Missing Values
The imputation will handle the numerical and categorical cleaned data separatley. Firstly, the categorical data will be imputed with (INSERT METHOD/S) which will (INSERT WHAT THEY WILL DO). Secondly, the numerical data will be imputated using (INSERT METHOD) (INSTERT WHAT IT WILL DO).

#### Imputating the Categorical Data

In [57]:
# Looping through the categorical features to check if they need imputation.
# If needed, then the loop will dynamically check which imputer to use based on the number of unique values.
for col in df_categorical.columns:
    if df_categorical[col].isnull().sum() > 0:

        if df_categorical[col].nunique() <= 2:
            print(f'Binary feature {col} with {df_categorical[col].nunique()} unique values')
            print('Using the most frequent imputer\n')
            imputer = SimpleImputer(strategy='most_frequent')

        elif df_categorical[col].nunique() < 10 and df_categorical[col].nunique() > 2:
            print(f'Categorical feature {col} with {df_categorical[col].nunique()} unique values')
            print('Checking spread of values to decide on imputer')
            print('...')

            # Check the spread of values and their ratio of missing values
            value_counts = df_categorical[col].value_counts(normalize=True)
            na_ratio = df_categorical[col].isna().mean()
            
            # Creating a decision tree for which imputer to use based on the spread of values and the ratio of missing values.
            if value_counts.max() > 0.6 and na_ratio <= 0.05:
                print('Using the most frequent imputer')
                imputer = SimpleImputer(strategy='most_frequent')
            else:
                print('Using the missing value imputer')
                imputer = SimpleImputer(strategy='constant', fill_value='missing')
        else:
            print(f'Continuous feature {col} with {df_categorical[col].nunique()} unique values')
            print('Using missing value imputer')
            imputer = SimpleImputer(strategy='constant', fill_value='missing')

        # Imputing the missing values
        df_categorical.loc[:, col] = imputer.fit_transform(df_categorical[[col]]).ravel()
        print('Imputation done\n')


Categorical feature race with 5 unique values
Checking spread of values to decide on imputer
...
Using the most frequent imputer
Imputation done



In [61]:
df_numerical.head(10)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,1,41,0,1,0,0,0,1
1,3,59,0,18,0,0,0,9
2,2,11,5,13,2,0,1,6
3,2,44,1,16,0,0,0,7
4,1,51,0,8,0,0,0,5
5,3,31,6,16,0,0,0,9
6,4,70,1,21,0,0,0,7
7,5,73,0,12,0,0,0,8
8,13,68,2,28,0,0,0,8
9,12,33,3,18,0,0,0,8


#### Imputating the Numerical Data

In [ ]:
# Check if imputation is done, in order to check if we should show the DataFrame or not.
counter = 0

for col in df_numerical.columns:
    if df_numerical[col].isnull().sum() > 0:
        print(f'Imputation needed for {col} with {df_categorical[col].nunique()} unique values')
        print('Using the median imputer')
        
        # Using the median imputer for the numerical features
        imputer = SimpleImputer(strategy='median')
        
        # Imputing the missing values
        df_numerical.loc[:, col] = imputer.fit_transform(df_numerical[[col]]).ravel()
        print('Imputation done\n')

        counter += 1
    else:
        print(f'No imputation needed for {col}.')

# Checking if it is worth showing the DataFrame or not. 
if counter != 0:
    df_numerical.head(10)

No imputation needed for time_in_hospital.
No imputation needed for num_lab_procedures.
No imputation needed for num_procedures.
No imputation needed for num_medications.
No imputation needed for number_outpatient.
No imputation needed for number_emergency.
No imputation needed for number_inpatient.
No imputation needed for number_diagnoses.


#### Final Preprocessing Checks
A final check to make sure that the performed imputation and cleaning results in a dataset that is free of missing values or reduantand, low-value features. 


#### Final Check of the categorical Data

In [74]:
# Some prints for manually checking how the imputation went
print('Exists NaN values:', df_categorical.isnull().sum().sum() > 0)
print('Shape of the cateogircal DataFrame: ', df_categorical.shape)
print('Other information about the dataFrame:\n', df_categorical.info())
print('Unique values per feature\n', df_categorical.nunique())


Exists NaN values: False
Shape of the cateogircal DataFrame:  (101766, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   race                      101766 non-null  object
 1   gender                    101766 non-null  object
 2   age                       101766 non-null  object
 3   metformin                 101766 non-null  object
 4   glipizide                 101766 non-null  object
 5   glyburide                 101766 non-null  object
 6   insulin                   101766 non-null  object
 7   change                    101766 non-null  object
 8   diabetesMed               101766 non-null  object
 9   readmitted                101766 non-null  object
 10  admission_type_id         101766 non-null  int64 
 11  discharge_disposition_id  101766 non-null  int64 
 12  admission_source_id       101766 non-nu

#### Final Check of the Numerical Data

In [75]:
# Some prints for manually checking how the imputation went
print('Exists NaN values:', df_numerical.isnull().sum().sum() > 0)
print('Shape of the numerical DataFrame: ', df_numerical.shape)
print('Other information about the dataFrame:\n', df_numerical.info())
print('Unique values per feature\n', df_numerical.nunique())
df_numerical.describe()


Exists NaN values: False
Shape of the numerical DataFrame:  (101766, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype
---  ------              --------------   -----
 0   time_in_hospital    101766 non-null  int64
 1   num_lab_procedures  101766 non-null  int64
 2   num_procedures      101766 non-null  int64
 3   num_medications     101766 non-null  int64
 4   number_outpatient   101766 non-null  int64
 5   number_emergency    101766 non-null  int64
 6   number_inpatient    101766 non-null  int64
 7   number_diagnoses    101766 non-null  int64
dtypes: int64(8)
memory usage: 6.2 MB
Other information about the dataFrame:
 None
Unique values per feature
 time_in_hospital       14
num_lab_procedures    118
num_procedures          7
num_medications        75
number_outpatient      39
number_emergency       33
number_inpatient       21
number_diagnoses       16
dtype: int64


,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,4.395987,43.095641,1.339730,16.021844,0.369357,0.197836,0.635566,7.422607
std,2.985108,19.674362,1.705807,8.127566,1.267265,0.930472,1.262863,1.933600
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000
75%,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000
max,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000


## 3. Target Variable Transformation 🎯

## 4. Identify Feature Types 🧬

## 5. Class Balance Check 📈

## 6. Feature Engineering 🧪

## 7. Encode Categorical Variables 🔧

## 8. Scale Numerical Variables ⚖️

## 9. Build ML Pipeline 🧱

## 10. Export Cleaned Data 🧼